In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
# google drive mount

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
# 데이터 불러오기

import pandas as pd

dataset = pd.read_table('/gdrive/My Drive/KoBERT/data/data_final.txt')

In [ ]:
dataset

,sentence,emotion
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,화남
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,비참함
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,비참함
3,꼭 롤러코스터 타는 것 같아요.,긴장됨
4,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,우울함
...,...,...
46105,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,괴로움
46106,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어.,기쁨
46107,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,우울함
46108,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야.,기쁨


In [ ]:
dataset.emotion.unique()

array(['화남', '비참함', '긴장됨', '우울함', '괴로움', '중립', '놀람', '지루함', '피로함', '기쁨'],
      dtype=object)

In [ ]:
# 감정 라벨링

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(dataset['emotion'])
dataset['emotion'] = encoder.transform(dataset['emotion'])
dataset.head()

,sentence,emotion
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,9
1,더 이상 내 감정을 내가 컨트롤 못 하겠어.,4
2,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,4
3,꼭 롤러코스터 타는 것 같아요.,2
4,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,5


In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '괴로움',
 1: '기쁨',
 2: '긴장됨',
 3: '놀람',
 4: '비참함',
 5: '우울함',
 6: '중립',
 7: '지루함',
 8: '피로함',
 9: '화남'}

In [ ]:
data_list = []
for q, label in zip(dataset['sentence'], dataset['emotion']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [ ]:
# train/test 분리

from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size=0.2, random_state=42)
print("train:", len(train))
print("test:", len(test))

train: 36888
test: 9222


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 10, # softmax 사용
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/577 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.3691534996032715 train acc 0.109375
epoch 1 batch id 201 loss 1.8819173574447632 train acc 0.22263681592039802
epoch 1 batch id 401 loss 1.5362106561660767 train acc 0.3514261221945137
epoch 1 train acc 0.4131914355863663


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/145 [00:00<?, ?it/s]

...

epoch 9 test acc 0.5977729885057471


  0%|          | 0/577 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.23370400071144104 train acc 0.953125
epoch 10 batch id 201 loss 0.22043892741203308 train acc 0.933768656716418
epoch 10 batch id 401 loss 0.2322445958852768 train acc 0.9357076059850374
epoch 10 train acc 0.9356856585788561


  0%|          | 0/145 [00:00<?, ?it/s]

epoch 10 test acc 0.5966954022988505


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("괴로움")
            elif np.argmax(logits) == 1:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 2:
                test_eval.append("긴장됨")
            elif np.argmax(logits) == 3:
                test_eval.append("놀람")
            elif np.argmax(logits) == 4:
                test_eval.append("비참함")
            elif np.argmax(logits) == 5:
                test_eval.append("우울함")
            elif np.argmax(logits) == 6:
                test_eval.append("중립")
            elif np.argmax(logits) == 7:
                test_eval.append("지루함")
            elif np.argmax(logits) == 8:
                test_eval.append("피로함")
            elif np.argmax(logits) == 9:
                test_eval.append("화남")

        print(">> 입력하신 내용에서 " + test_eval[0] + "이 느껴집니다.\n")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    input_sentence = input("챗봇에게 하고싶은 말을 입력해주세요 : ")
    if input_sentence == 0 :
        break
    predict(input_sentence)
    # sentence = input_sentence.split('.')
    # for i in range(len(sentence)):
    #   predict(sentence[i])
    print("\n")

챗봇에게 하고싶은 말을 입력해주세요 : 아 진짜 피곤하다
>> 입력하신 내용에서 피로함이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 오늘 너무 힘들었어. 빨리 집 가서 쉬어야지
>> 입력하신 내용에서 화남이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 죽고싶다
>> 입력하신 내용에서 괴로움이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 죽고싶어
>> 입력하신 내용에서 우울함이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 오늘 아침부터 바쁘게 움직였더니 너무 힘드네
>> 입력하신 내용에서 괴로움이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 종료
>> 입력하신 내용에서 기쁨이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 1
>> 입력하신 내용에서 중립이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 1
>> 입력하신 내용에서 중립이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 0
>> 입력하신 내용에서 중립이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : '0'
>> 입력하신 내용에서 비참함이 느껴집니다.



챗봇에게 하고싶은 말을 입력해주세요 : 0
>> 입력하신 내용에서 중립이 느껴집니다.





KeyboardInterrupt: ignored

In [ ]:
# 테스트 문장 예측
test_sentence = ''
test_label = 7 # 실제 감정

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['sentence', 'emotion']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)